In [ ]:
%load_ext autoreload
%autoreload 2
import os
import numpy as np
import pandas as pd

import tensorflow as tf
from micron2.clustering import Autoencoder, train_AE_simCLR
from micron2.data import stream_dataset

import tqdm.auto as tqdm

outdir = 'trained_simclr'

In [ ]:
import h5py
f = h5py.File('/dev/shm/dataset.hdf5', 'r')
print(f['meta/channel_names'][:])
use_channels = [b.decode('UTF-8') for b in f['meta/channel_names'][:]]
# fn = lambda x,y: print(x)
print(f.keys())
print(f['cells'].keys())
f.close()

In [ ]:
sample_x = tf.zeros((8, 64, 64, len(use_channels)), dtype=tf.float32)
ae_model = Autoencoder(input_shape=sample_x.shape[1:])
y, z_g = ae_model(sample_x, return_g=True)
print(y.shape)
print(z_g.shape)
z = ae_model.encode(sample_x)
print(z.shape)

In [ ]:
ae_model.load_weights(f'{outdir}/weights.h5')

In [ ]:
def process(x):
    x = tf.cast(x, tf.float32)/255.
    x = tf.transpose(tf.image.per_image_standardization(tf.transpose(x)))
    return x
    
# use_channels=['DAPI', 'CD45', 'PanCytoK']
# use_channels=['DAPI', 'CD45', 'PanCytoK', 'CD31', 'PDGFRb', 'aSMA', 'Ki-67']
dataset = stream_dataset('/dev/shm/dataset.hdf5', use_channels = use_channels)
dataset = (dataset.map(process)
           .batch(64)
          )

z = []
z_L2 = []
for batch in tqdm.tqdm(dataset):
    # batch = tf.image.random_crop(batch, size=(batch.shape[0], 48, 48, batch.shape[-1]))
    z.append(ae_model.encode_g(batch).numpy().copy())
    z_L2.append(ae_model.encode(batch).numpy().copy())
    
z = np.concatenate(z, axis=0)
z_L2 = np.concatenate(z_L2, axis=0)
print(z.shape)
print((z.sum(0)==0).sum())

print(z_L2.shape)
print((z_L2.sum(0)==0).sum())

In [ ]:
np.save(f'{outdir}/z.npy', z)
np.save(f'{outdir}/z_L2.npy', z_L2)

In [ ]:
!ls -lha trained_simclr

In [ ]:
(140000 / 256) * 5